<a href="https://colab.research.google.com/github/lizhenbasel/AI_2022/blob/main/StyleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex torch==1.7.1 torchvision==0.8.2

/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 16.0 MB/s 
     |████████████████████████████████| 776.8 MB 4.8 kB/s 
     |████████████████████████████████| 12.8 MB 64.8 MB/s 
     |████████████████████████████████| 17.3 MB 65.0 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0+cu116
    Uninstalling torch-1.13.0+cu116:
      Successfully uninstalled torch-1.13.0+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.0+cu116
    Uninstalling torchvision-0.14.0+cu116:
      Successfully uninstalled torchvision-0.14.0+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages

In [3]:
%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py generate.py legacy.py closed_form_factorization.py flesh_digression.py apply_factor.py README.md calc_metrics.py training/stylegan2_multi.py training/training_loop.py util/utilgan.py

/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch
Already up to date.
Saved working directory and index state WIP on main: 59e05bb added opensimplex


## Making a dataset

In [4]:
export_dir = '/content/drive/MyDrive/SimpleFoods'

In [5]:
import errno

In [6]:
# square crop and resize to 512x512

import PIL
import os
import os.path
from PIL import Image

newsize = (512, 512)

square_dir = "/content/square"
try:
    os.makedirs(square_dir)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

def crop_max_square(pil_img):
    return crop_center(pil_img, min(pil_img.size), min(pil_img.size))

f = export_dir

for file in os.listdir(f):
    f_img = f+"/"+file
    im = Image.open(f_img)
    
    im = im.convert('RGB') # convert to rgb
    
    
    im_square = crop_max_square(im)

    im_square = im_square.resize(newsize)

    im_square.save(square_dir+"/"+file)

In [7]:
os.path.isdir('/content/square')

True

In [8]:
# check size and colour format of images
from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm.notebook import tqdm

IMAGE_PATH = '/content/square'
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz

  0%|          | 0/509 [00:00<?, ?it/s]

In [9]:
# zip folder
!zip -r /content/square.zip /content/square > /dev/null

## IMPORTING THE DATASET

In [10]:
import zipfile
with zipfile.ZipFile('/content/square.zip') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/food_square')

In [11]:
try:
    os.makedirs('./datasets/square_images')
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
!python /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/dataset_tool.py --source /content/square --dest /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/datasets/square_images

Error: --dest folder must be empty


In [ ]:
!mv '/content/drive/MyDrive/minecraft-gan-2020-12-22.pkl' '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained'

mv: cannot stat '/content/drive/MyDrive/minecraft-gan-2020-12-22.pkl': No such file or directory


# Train model
Below are a series of variables you need to set to run the training.

**dataset_path**: this is the path to your folder

**resume_from**: if you’re starting a new dataset I recommend 'ffhq1024' or './pretrained/wikiart.pkl'

**mirror_x and mirror_y**: Allow the dataset to use horizontal or vertical mirroring.

Make sure you start aug_strength and train_count from zero. Then will then need updating each time you restart the training.

In [ ]:
#required: definitely edit these!
dataset_path = '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/datasets/square_images/00000'
#resume_from = './pretrained/FreaGAN.pkl'
#resume_from = './pretrained/wikiart.pkl'
#resume_from = './pretrained/2020-01-11-skylion-stylegan2-animeportraits-networksnapshot-024664.pkl'
resume_from = './pretrained/minecraft-gan-2020-12-22.pkl'

aug_strength = 0.0
train_count = 0
#ug_strength = 0.502
#train_count = 168

mirror_x = True
#mirror_y = False

#optional: you might not need to edit these
gamma_value = 50.0
augs = 'bg'
config = '11gb-gpu'
snapshot_count = 1

In [ ]:
!python train.py --gpus=1 --cfg=$config --metrics=None --outdir=./results --data=$dataset_path --snap=$snapshot_count --resume=$resume_from --augpipe=$augs --initstrength=$aug_strength --gamma=$gamma_value --mirror=$mirror_x --mirrory=False --nkimg=$train_count

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"

Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 1,
  "network_snapshot_ticks": 1,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/datasets/square_images/00000",
    "use_labels": false,
    "max_size": 509,
    "xflip": true,
    "resolution": 512
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max

In [ ]:
!python generate.py --outdir=/content/out/images/ --trunc=0.8 --seeds=10 --network='./pretrained/minecraft-gan-2020-12-22.pkl'

generate.py:59: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  elif(len(seeds) is not 3):
Traceback (most recent call last):
  File "generate.py", line 25, in <module>
    from opensimplex import OpenSimplex
  File "/usr/local/lib/python3.8/dist-packages/opensimplex/__init__.py", line 4, in <module>
    from .api import *
  File "/usr/local/lib/python3.8/dist-packages/opensimplex/api.py", line 2, in <module>
    from .internals import _init, _noise2, _noise3, _noise4, _noise2a, _noise3a, _noise4a
  File "/usr/local/lib/python3.8/dist-packages/opensimplex/internals.py", line 9, in <module>
    from numba import njit, prange
  File "/usr/local/lib/python3.8/dist-packages/numba/__init__.py", line 42, in <module>
    from numba.np.ufunc import (vectorize, guvectorize, threading_layer,
  File "/usr/local/lib/python3.8/dist-packages/numba/np/ufunc/__init__.py", line 3, in <module>
    from numba.np.ufunc.decorators import Vectorize, GUVectorize, vectorize, guvectorize
  File "/